# How to Predict Multiple Time Series At Once With Scikit-Learn in 10 Simple Steps
## Sales Forecast Case Study
### Mario Filho - mariofilho.com

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

# 1. Melt the data, stack the series

In [2]:
data = pd.read_csv('Sales_Transactions_Dataset_Weekly.csv')
data = data.filter(regex=r'Product|W')
data.head()

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,P1,11,12,10,8,13,12,14,21,6,...,4,7,8,10,12,3,7,6,5,10
1,P2,7,6,3,2,7,1,6,3,3,...,2,4,5,1,1,4,5,1,6,0
2,P3,7,11,8,9,10,8,7,13,12,...,6,14,5,5,7,8,14,8,8,7
3,P4,12,8,13,5,9,6,9,13,13,...,9,10,3,4,6,8,14,8,7,8
4,P5,8,5,13,11,6,7,9,14,9,...,7,11,7,12,6,6,5,11,8,9


In [3]:
melt = data.melt(id_vars='Product_Code', var_name='Week', value_name='Sales')
melt.head()

,Product_Code,Week,Sales
0,P1,W0,11
1,P2,W0,7
2,P3,W0,7
3,P4,W0,12
4,P5,W0,8


In [4]:
melt['Product_Code'] = melt['Product_Code'].str.extract('(\d+)', expand=False).astype(int)
melt['Week'] = melt['Week'].str.extract('(\d+)', expand=False).astype(int)

melt = melt.sort_values(['Week', 'Product_Code'])
melt.head()

,Product_Code,Week,Sales
0,1,0,11
1,2,0,7
2,3,0,7
3,4,0,12
4,5,0,8


# 2. Split the data

In [5]:
split_point = 40
melt_train = melt[melt['Week'] < split_point].copy()
melt_valid = melt[melt['Week'] >= split_point].copy()

In [6]:
melt_train.head()

,Product_Code,Week,Sales
0,1,0,11
1,2,0,7
2,3,0,7
3,4,0,12
4,5,0,8


# 3. Set up a 1-step target

In [7]:
melt_train['sales_next_week'] = melt_train.groupby("Product_Code")['Sales'].shift(-1)

In [8]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week
0,1,0,11,12.0
811,1,1,12,10.0
1622,1,2,10,8.0
2433,1,3,8,13.0
3244,1,4,13,12.0


In [9]:
melt_valid['sales_next_week'] = melt_valid.groupby("Product_Code")['Sales'].shift(-1)

In [10]:
melt_train.tail()

,Product_Code,Week,Sales,sales_next_week
32435,815,39,2,NaN
32436,816,39,6,NaN
32437,817,39,0,NaN
32438,818,39,0,NaN
32439,819,39,0,NaN


In [11]:
melt_train = melt_train.dropna()

In [12]:
melt_train.tail()

,Product_Code,Week,Sales,sales_next_week
31624,815,38,1,2.0
31625,816,38,4,6.0
31626,817,38,0,0.0
31627,818,38,0,0.0
31628,819,38,1,0.0


# 4. Create 4 Fundamental Features

## 4.1 Lag
- what if I don't have the last period? Get the closest available

In [13]:
melt_train["lag_sales_1"] = melt_train.groupby("Product_Code")['Sales'].shift(1)

In [14]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1
0,1,0,11,12.0,NaN
811,1,1,12,10.0,11.0
1622,1,2,10,8.0,12.0
2433,1,3,8,13.0,10.0
3244,1,4,13,12.0,8.0


In [15]:
melt_valid["lag_sales_1"] = melt_valid.groupby("Product_Code")['Sales'].shift(1)

## 4.2 Difference

In [16]:
melt_train["diff_sales_1"] = melt_train.groupby("Product_Code")['Sales'].diff(1)

In [17]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1
0,1,0,11,12.0,NaN,NaN
811,1,1,12,10.0,11.0,1.0
1622,1,2,10,8.0,12.0,-2.0
2433,1,3,8,13.0,10.0,-2.0
3244,1,4,13,12.0,8.0,5.0


In [18]:
melt_valid["diff_sales_1"] = melt_valid.groupby("Product_Code")['Sales'].diff(1)

## 4.3 Rolling statistics
- Mean
- Max
- Min
- Std

In [19]:
melt_train.groupby("Product_Code")['Sales'].rolling(4).mean()

Product_Code       
1             0          NaN
              811        NaN
              1622       NaN
              2433     10.25
              3244     10.75
                       ...  
819           28384     0.25
              29195     1.00
              30006     1.00
              30817     1.25
              31628     1.50
Name: Sales, Length: 31629, dtype: float64

In [20]:
melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

0          NaN
811        NaN
1622       NaN
2433     10.25
3244     10.75
         ...  
28384     0.25
29195     1.00
30006     1.00
30817     1.25
31628     1.50
Name: Sales, Length: 31629, dtype: float64

In [21]:
melt_train["mean_sales_4"] = melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

In [22]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
0,1,0,11,12.0,NaN,NaN,NaN
811,1,1,12,10.0,11.0,1.0,NaN
1622,1,2,10,8.0,12.0,-2.0,NaN
2433,1,3,8,13.0,10.0,-2.0,10.25
3244,1,4,13,12.0,8.0,5.0,10.75


In [23]:
melt_valid["mean_sales_4"] = melt_valid.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

# 4.4 Date components (seasonality) - not available in this dataset
- Day of week
- Day of year
- Month
- Day of month

### Extra tip: what about interactions? (e.g.: Lag 1 - Rolling 7)

# 5. Choose an evaluation metric
- MAPE: https://en.wikipedia.org/wiki/Mean_absolute_percentage_error
    - zeros?
- WMAPE: https://en.wikipedia.org/wiki/WMAPE

In [24]:
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    #ape[~np.isfinite(ape)] = 0. # VERY questionable
    ape[~np.isfinite(ape)] = 1. # pessimist estimate
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# 6. Establish baseline
- Predict next week sales as equal to this week sales

In [25]:
y_pred = melt_train['Sales']
y_true = melt_train['sales_next_week']

In [26]:
mape(y_true, y_pred)

0.6721872645511404

In [27]:
wmape(y_true, y_pred)

0.30816465612331645

# 7. Train the model

In [28]:
melt_train.head(5)

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
0,1,0,11,12.0,NaN,NaN,NaN
1,2,0,7,6.0,NaN,NaN,NaN
2,3,0,7,11.0,NaN,NaN,NaN
3,4,0,12,8.0,NaN,NaN,NaN
4,5,0,8,5.0,NaN,NaN,NaN


In [29]:
features = ['Sales', 'lag_sales_1', 'diff_sales_1', 'mean_sales_4']

In [30]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(melt_train[features])
ytr = melt_train['sales_next_week']


mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
mdl.fit(Xtr, ytr)

RandomForestRegressor(n_jobs=6, random_state=0)

# 8. Evaluate the model
- Did we beat the baseline?

In [31]:
Xval = imputer.transform(melt_valid[features])
yval = melt_valid['sales_next_week']

p = mdl.predict(Xval)

In [32]:
mape(yval, p)

0.6463137461455405

In [33]:
wmape(yval, p)

0.3004969729507602

# 9. Extend the model to predict n-steps

In [34]:
melt_train['sales_next_next_week'] = melt_train.groupby("Product_Code")['Sales'].shift(-2)
melt_valid['sales_next_next_week'] = melt_valid.groupby("Product_Code")['Sales'].shift(-2)

In [35]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
0,1,0,11,12.0,NaN,NaN,NaN,10.0
811,1,1,12,10.0,11.0,1.0,NaN,8.0
1622,1,2,10,8.0,12.0,-2.0,NaN,13.0
2433,1,3,8,13.0,10.0,-2.0,10.25,12.0
3244,1,4,13,12.0,8.0,5.0,10.75,14.0


In [36]:
melt_train = melt_train.dropna(subset=['sales_next_week','sales_next_next_week'])

In [37]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(melt_train[features])
ytr = melt_train[['sales_next_week', 'sales_next_next_week']]

mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
mdl.fit(Xtr, ytr)

RandomForestRegressor(n_jobs=6, random_state=0)

In [38]:
Xval = imputer.transform(melt_valid[features])
yval = melt_valid[['sales_next_week', 'sales_next_next_week']]

p = mdl.predict(Xval)

In [39]:
mape(yval, p)

sales_next_week         0.647034
sales_next_next_week    0.681146
dtype: float64

In [40]:
wmape(yval, p)

sales_next_week         0.300301
sales_next_next_week    0.310315
dtype: float64

# 10. Predicting new examples
- As long as you have the same features you used to train, you can predict for any period

In [41]:
melt_valid.tail()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
42167,815,51,0,NaN,2.0,-2.0,0.50,NaN
42168,816,51,5,NaN,6.0,-1.0,5.25,NaN
42169,817,51,3,NaN,4.0,-1.0,1.75,NaN
42170,818,51,0,NaN,2.0,-2.0,0.50,NaN
42171,819,51,1,NaN,0.0,1.0,0.25,NaN


In [42]:
new_examples = melt_valid[melt_valid['Week'] == 51].copy()
new_examples.head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
41361,1,51,10,NaN,5.0,5.0,7.00,NaN
41362,2,51,0,NaN,6.0,-6.0,3.00,NaN
41363,3,51,7,NaN,8.0,-1.0,9.25,NaN
41364,4,51,8,NaN,7.0,1.0,9.25,NaN
41365,5,51,9,NaN,8.0,1.0,8.25,NaN


In [43]:
p = mdl.predict(new_examples[features])
p

array([[ 4.70014286,  8.85766667],
       [ 1.69609452,  3.01583297],
       [10.90119048,  5.80107143],
       ...,
       [ 0.74560423,  2.15993741],
       [ 0.52880028,  0.49030189],
       [ 0.27852871,  0.30830163]])

In [44]:
new_examples['p_sales_next_week'] = p[:, 0]
new_examples['p_sales_next_next_week'] = p[:, 1]

In [45]:
new_examples.head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week,p_sales_next_week,p_sales_next_next_week
41361,1,51,10,NaN,5.0,5.0,7.00,NaN,4.700143,8.857667
41362,2,51,0,NaN,6.0,-6.0,3.00,NaN,1.696095,3.015833
41363,3,51,7,NaN,8.0,-1.0,9.25,NaN,10.901190,5.801071
41364,4,51,8,NaN,7.0,1.0,9.25,NaN,8.492500,13.952071
41365,5,51,9,NaN,8.0,1.0,8.25,NaN,9.381560,8.619524


# Next Steps
- Try more features with more periods
- Try different models
- Find which products have the worst errors and check if there is a fixable pattern
- Tune the model

# Your turn!
## Apply what you learned here in your dataset and share the results with our community :)